In [2]:
#import modules
import pandas as pd
import numpy as np
import requests

from pandas.io import gbq
#from google.cloud import bigquery,storage
from google.oauth2 import service_account

import gspread
import json, os
from canvasapi import Canvas

from datetime import datetime

from time import time

from params import basedir, run_mode
#import plotly.express as px

In [3]:
#choose credential file paths and other possible changes:
#run_mode = 'dev' #or, 'prod', or 'mig' for when migrating the code
print(run_mode)

dev


In [4]:

if run_mode == 'dev':
    out_table = "all_courses3"
    tab_out_table = "all_course_tabs3"
    
elif run_mode == 'prod':
    out_table = "all_courses4"
    tab_out_table = "all_course_tabs4"    

elif run_mode == 'prod_home':
    out_table = "all_courses4"
    tab_out_table = "all_course_tabs4"    

    
elif run_mode == 'mig':
    out_table = "all_courses3"
    tab_out_table = "all_course_tabs3"    



In [5]:

#get Google cloud credentials
project_id = 'canvas-portal-data-custom'
cred_file = '{}/canvas-portal-data-custom-6e244db3b826.json'.format(basedir)
data_dl = 'data'
scopes = [ "https://www.googleapis.com/auth/drive", "https://www.googleapis.com/auth/drive.file",
            "https://spreadsheets.google.com/auth/spreadsheets"]
credentials = service_account.Credentials.from_service_account_file(cred_file,)

#get Canvas credentials
#cred_file2 = '{}/instances.json'.format(basedir)
#with open(cred_file2,'r') as cred2:
#    cred_json = json.load(cred2)

from instances import canvas_api, panopto_api

canvas_API_KEY = canvas_api['ACCES_TOKEN']
canvas_API_URL = canvas_api['API_URL']#+'/accounts/1'

panopto_client_id = panopto_api['Client Id']
panopto_client_secret = panopto_api['Client Secret']

In [6]:
# Initialize a new Canvas object
canvas = Canvas(canvas_API_URL, canvas_API_KEY)
canvas.__dict__

#Initialize a new Panopto object

{'_Canvas__requester': <canvasapi.requester.Requester at 0x7fd9e949fa60>}

In [27]:
#Get a list of all department names by the sub-account id:
acc = canvas.get_account(1)
sub_account_dict = {}
sub_account_dict[1] = 'MIT Root Account'
parent_account_dict = {}
accs = acc.get_subaccounts(recursive=True)
for a_ in accs:
    sub_account_dict[a_.id] = a_.name

for a_ in accs:
    try:
        parent_account_dict[a_.id] = sub_account_dict[a_.parent_account_id]
    except Exception as e:
        pass

    
#Get course terms associated with the account and create a dictionary of the term names:
term_dict ={}
for term_ in acc.get_enrollment_terms():
    term_dict[term_.id] = term_.name
    
#parent_account_dict
#sub_account_dict

In [13]:
#Get a list of all courses:
print("Getting a list of all courses...")
large_num = 100000 #give a very large number so all courses are pulled recursively
#excl_acc = [1,17,76] #mention sub-accounts to be excluded
excl_acc = [] #Don't implement it here...
begin_date = '2015-05-01' #Date since when we are counting

course_rows = []
course_cols = ['course_id', 'course_code', 'course_name', 'account_id', 'created_at']
for acc_course in acc.get_courses()[:large_num]:
    course_rows.append([acc_course.id, acc_course.course_code, acc_course.name, acc_course.account_id, 
                        acc_course.created_at])
    
courses_df = pd.DataFrame.from_records(course_rows, columns=course_cols)

courses_df_dep_excluded = courses_df[~courses_df.account_id.isin(excl_acc)]
'''
try:
    courses_df_dep_excluded['Dept'] = courses_df_dep_excluded.apply(lambda 
                                                        row: sub_account_dict[row['account_id']], axis=1)
except:
    pass
courses_df_dep_excluded
'''
filtered_courses_df = courses_df_dep_excluded[courses_df_dep_excluded.created_at>=begin_date]

filtered_courses_df.to_csv('all_canvas_courses.csv', index=None)
print("Getting a list of all courses... Done.")

Getting a list of all courses...
Getting a list of all courses... Done.


In [36]:
def get_course_tab_info(course_id):
    '''This function gets the course information and the file/assignment update times,
    by the course_id, and returns a list of '''
    cutoff_date = datetime.strptime('2015-05-31', '%Y-%m-%d') #by this time most default content was created
    tab_dict_course = []
    try:
        c1 = canvas.get_course(course_id)
    except:
        c1 = None    
    try:    
        #Get all tabs for this course
        all_tabs = c1.get_tabs()

        #Append the list of tabs to all_tabs_list
        for tab_ in all_tabs:
            tab_dict = tab_.__dict__
            tab_dict['course_name'] = c1.name

            tab_dict_course.append(tab_dict)

        return tab_dict_course
    except:
        pass

In [37]:
#get the course tab info for a set number of courses at a time
filtered_courses_df = pd.read_csv('all_canvas_courses.csv')
all_course_list_0 = filtered_courses_df.course_id.tolist()


if run_mode == 'dev' or run_mode=='mig':
    course_pos_list = all_course_list_0[:5]
    num_courses_dump = 3
elif run_mode == 'prod' or run_mode=='prod_home':
    course_pos_list = all_course_list_0
    num_courses_dump = 100
    
course_max_num = len(course_pos_list)
tab_dict_list = []

dump_num = 0
i = 0
while i < course_max_num:
    cid = course_pos_list[i]
    tab_dict_course = get_course_tab_info(cid)
    for item in tab_dict_course:
        tab_dict_list.append(item)


    if  i%num_courses_dump == 0:
        course_tab_df = pd.DataFrame(tab_dict_list)
        course_tab_df.drop(['_requester'], axis=1, inplace=True)
        if dump_num==0: 
            course_tab_df.to_csv('course_tab_df.csv', index=None)

        else: 
            course_tab_df.to_csv('course_tab_df.csv', index=None, header=None, mode='a')

        dump_num += 1 #increment the dump number
        tab_dict_list = [] #start a new tab_dict_list
        
    i += 1 #move course position in the course list        

#finally, once out of the loop, check if the tab_dict_list is empty, and dump it if not.
if len(tab_dict_list) != 0:
        course_tab_df = pd.DataFrame(tab_dict_list)
        course_tab_df.drop(['_requester'], axis=1, inplace=True)
        if dump_num==0: 
            course_tab_df.to_csv('course_tab_df.csv', index=None)
        else: 
            course_tab_df.to_csv('course_tab_df.csv', index=None, header=None, mode='a')

        
        
    


In [38]:
#Create course tab dataframe from the list:
course_tab_df = pd.read_csv('course_tab_df.csv')

#Send out course tabs dataframe to GBQ
course_tab_df.to_gbq('lt_courses.{}'.format(tab_out_table), project_id, if_exists='replace', credentials=credentials)

1it [00:04,  4.38s/it]
